# Imports

In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import torch.nn.functional as F

Using TensorFlow backend.


# Padding and One-Hot Encoding

In [2]:
#Extract list of smile strings from data
data = pd.read_csv('logp_data.csv')
smiles = data['smiles'].values

In [3]:
#Pad text
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(smiles)
seq_of_ints = tokenizer.texts_to_sequences(smiles)

In [5]:
import pickle

f = open("vocab_mapping.pkl", 'wb')

tokenizer.word_index

{'C': 1,
 '(': 2,
 ')': 3,
 '1': 4,
 'O': 5,
 '@': 6,
 'N': 7,
 'c': 8,
 '[': 9,
 ']': 10,
 'H': 11,
 '=': 12,
 'n': 13,
 '2': 14,
 '#': 15,
 'F': 16,
 'S': 17,
 'o': 18,
 '/': 19,
 '-': 20,
 '3': 21,
 's': 22,
 '\\': 23,
 '+': 24,
 'l': 25,
 '4': 26,
 'B': 27,
 'P': 28,
 'r': 29,
 '5': 30,
 'i': 31,
 '6': 32,
 'I': 33,
 '7': 34,
 '8': 35,
 'p': 36}

In [4]:
MAX_WORD = len(tokenizer.word_counts)
for i in range(len(seq_of_ints)):
    seq_of_ints[i] = [MAX_WORD + 1] + seq_of_ints[i] + [MAX_WORD + 2]

In [5]:
#One hot encode
seq_of_ints = torch.LongTensor(pad_sequences(seq_of_ints, padding='post', maxlen=75))

In [11]:
y = seq_of_ints[:, 1:]

In [13]:
y.shape

torch.Size([621193, 74])

In [14]:
final_data = F.one_hot(seq_of_ints)[:, :, 1:]
X = final_data[:, :-1, :]

In [19]:
X[0][0]

38

In [23]:
#Extract logP targets
y = torch.LongTensor(data['logp'].values)

# Splitting Data

In [24]:
len_of_data = len(smiles)
train_size = int(.8 * len_of_data)
test_size = len_of_data - train_size

In [25]:
#Load dataset into DataLoader
dataset = TensorDataset(X, y)
train_data, test_data = torch.utils.data.random_split(dataset, [train_size, test_size])
final_dataset = DataLoader(dataset, batch_size=32, shuffle=True)

In [30]:
train_data